# Sequence Operations: Review

### Sequence blocks are simple matrix operations:
- **RF Excitation**  
  $$
  M' = R_{\phi}(\alpha, t) M
  $$

- **Gradient-induced Rotation**  
  $$
  M' = R_z(\gamma \mathbf{G}(t) \cdot \mathbf{r} + \gamma \Delta B_0) M
  $$

- **Relaxation**  
  $$
  M' = A(\tau, T_1, T_2) M + B(\tau, T_1, T_2)
  $$

### Additional Notes:
- **Multiple spins, at different positions, can be simulated with loops.**


# Steady States with 3 Components

- **Propagation over 1 TR:**  
  $$
  M_{n+1} = A M_n + B
  $$

- **Steady State:**  
  $$
  M_{n+1} = M_n
  $$

- **Combine:**  
  $$
  M_{ss} = A M_{ss} + B \quad = \quad (I - A)^{-1} B
  $$

  - If there is relaxation, there is a steady state.


# Transients

- **Propagation over 1 TR:**  
  $$
  M_{n+1} = A M_n + B
  $$

- **Steady State:**  
  $$
  M_{ss} = A M_{ss} + B
  $$

- **Transient:**  
  $$
  M_{n+1} - M_{ss} = A (M_n - M_{ss}) = A^{n+1} (M_0 - M_{ss})
  $$

- **Eigenvector decomposition:**  
  $$
  A = V \Lambda V^{-1}
  $$

- **Write:**  
  $$
  M_{n+1} - M_{ss} = V \Lambda^{n+1} V^{-1} (M_0 - M_{ss})
  $$


In [42]:
%reload_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.join('..'))
import numpy as np
from scipy.spatial.transform import Rotation as R
from src import utils

![image.png](../imgs/short_tr_ir_signal.png)

# Inversion Recovery Sequence

- **TR = 1s, TI = 0.5s, TE = 50ms**
- **What is the signal for** $T_1 = 0.5s$, $T_2 = 100ms$?

### **"Operations"**
- $M_{180} = R_x(180) M_{TR}$
- $M_{90} = R_x(90) E(0.5s) M_{180}$
- $M_{TE} = E(0.05s) M_{90}$
- $M_{TR} = E(0.45s) = 1 - 0.5 - 0.05s) M_{TE}$


In [5]:
T1 = 0.5
T2 = 0.1

def E1(t):
    return np.exp(-t/T1)

def E2(t):
    return np.exp(-t/T2)

In [25]:
tau1 = 0.5
A1 = np.diag([E2(tau1), E2(tau1), E1(tau1)]) @ utils.rot_x(-180)
B1 = np.array([0,0,1-E1(tau1)])

tau2 = 0.05
A2 = np.diag([E2(tau2), E2(tau2), E1(tau2)]) @ utils.rot_x(-90)
B2 = np.array([0,0,1-E1(tau2)])

tau3 = 0.45
A3 = np.diag([E2(tau3), E2(tau3), E1(tau3)])
B3 = np.array([0,0,1-E1(tau3)])

In [ ]:
A = A2 @ A1 @ A3
B = B2 + A2 @ (B1 + A1 @ B3)
Mss = np.linalg.inv(np.eye(3) - A) @ B
A, B, Mss

In [ ]:
utils.abprop(A3, B3, A1, B1, A2, B2)